# Test SSO Login

This notebook will test the authorization of the notebooks running in this library.

By including the `jupyterauth` module, the notebook will initialize the authorization of the notebooks. 
* For instance profile / machine configurations, the role details are displayed to the screen.
* For SSO configurations, the AWS SSO login will appear asking the user to grant access to the session.
* For role assumption, the default profile is set


In [1]:
import boto3, os
from jupyterirtools import jupyterauth


Using default profile
Account: 383086473915, principal: sso-jupyter-al2-2-ExecutionRole-1U9TIN2DELAUB role session: SageMaker


The next cell will check all the configured permissions for this notebook. If the environment is configured for multiaccount via role assumption or sso, this will check each of the connections. 

In [2]:
jupyterauth.check_permissions()

### Account Status

|status|Account|Role|Status|
|-----|-----|-----|-----|
|<span style="color:green"> &#9679;</span>|383086473915|sso-jupyter-al2-2-ExecutionRole-1U9TIN2DELAUB|Successful|


This demonstrates how to use the `jupyterauth` module to iterate all the accounts and execute a boto command against every account. Here it is simply getting caller identity on each account.

In [3]:
for session, account in jupyterauth.get_session_by_account():
    sts = session.client('sts')
    identity = sts.get_caller_identity()
    print(f'Account: {account} User: {identity["UserId"]}')

Account: 383086473915 User: AROAVSMN3WK56E2HPBZKQ:SageMaker


The default session is configured based on the environment variables.

In [4]:
boto3.setup_default_session()
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROAVSMN3WK56E2HPBZKQ:SageMaker
Account: 383086473915



In [5]:
session =jupyterauth.get_session()
sts = session.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROAVSMN3WK56E2HPBZKQ:SageMaker
Account: 383086473915



Works with CLI commands too.

In [6]:
! aws sts get-caller-identity


{
    "UserId": "AROAVSMN3WK56E2HPBZKQ:SageMaker",
    "Account": "383086473915",
    "Arn": "arn:aws:sts::383086473915:assumed-role/sso-jupyter-al2-2-ExecutionRole-1U9TIN2DELAUB/SageMaker"
}


In [ ]:
jupyterauth.logout()